In [2]:
import pandas as pd

df = pd.read_csv('../data/external/hw1-data.csv')


In [3]:
df

,User,"Gender (1 =F, 0=M)",260: Star Wars: Episode IV - A New Hope (1977),1210: Star Wars: Episode VI - Return of the Jedi (1983),356: Forrest Gump (1994),"318: Shawshank Redemption, The (1994)","593: Silence of the Lambs, The (1991)",3578: Gladiator (2000),1: Toy Story (1995),2028: Saving Private Ryan (1998),...,2396: Shakespeare in Love (1998),2916: Total Recall (1990),780: Independence Day (ID4) (1996),541: Blade Runner (1982),1265: Groundhog Day (1993),"2571: Matrix, The (1999)",527: Schindler's List (1993),"2762: Sixth Sense, The (1999)",1198: Raiders of the Lost Ark (1981),34: Babe (1995)
0,755,0,1.0,5.0,2.0,NaN,4.0,4.0,2.0,2.0,...,2.0,NaN,5.0,2.0,NaN,4.0,2.0,5.0,NaN,NaN
1,5277,0,5.0,3.0,NaN,2.0,4.0,2.0,1.0,NaN,...,3.0,2.0,2.0,NaN,2.0,NaN,5.0,1.0,3.0,NaN
2,1577,1,NaN,NaN,NaN,5.0,2.0,NaN,4.0,NaN,...,NaN,1.0,4.0,4.0,1.0,1.0,2.0,3.0,1.0,3.0
3,4388,0,NaN,3.0,NaN,NaN,NaN,1.0,2.0,3.0,...,NaN,4.0,1.0,3.0,5.0,NaN,5.0,1.0,1.0,2.0
4,1202,1,4.0,3.0,4.0,1.0,4.0,1.0,NaN,4.0,...,5.0,1.0,NaN,4.0,NaN,3.0,5.0,5.0,NaN,NaN
5,3823,1,2.0,4.0,4.0,4.0,NaN,NaN,3.0,1.0,...,5.0,2.0,4.0,NaN,1.0,NaN,NaN,3.0,NaN,2.0
6,5448,0,NaN,NaN,3.0,1.0,1.0,4.0,NaN,5.0,...,1.0,NaN,NaN,3.0,NaN,1.0,NaN,NaN,5.0,2.0
7,5347,0,4.0,NaN,NaN,NaN,3.0,2.0,2.0,NaN,...,NaN,2.0,3.0,2.0,4.0,NaN,1.0,3.0,5.0,NaN
8,4117,1,5.0,1.0,NaN,4.0,2.0,4.0,4.0,4.0,...,2.0,3.0,1.0,NaN,5.0,NaN,NaN,NaN,NaN,5.0
9,2765,0,4.0,2.0,NaN,5.0,3.0,NaN,4.0,3.0,...,NaN,NaN,2.0,NaN,NaN,2.0,5.0,1.0,NaN,NaN


In [ ]:
# Calculate the mean for each movie, ignoring NaNs
mean_ratings = df.iloc[:, 2:].mean().sort_values(ascending=False)
mean_ratings.head(3)


318: Shawshank Redemption, The (1994)             3.600000
260: Star Wars: Episode IV - A New Hope (1977)    3.266667
541: Blade Runner (1982)                          3.222222
dtype: float64

In [13]:
num_ratings = df.iloc[:, 2:].count().sort_values(ascending=False)
num_ratings.head(3)

1: Toy Story (1995)                               17
593: Silence of the Lambs, The (1991)             16
260: Star Wars: Episode IV - A New Hope (1977)    15
dtype: int64

In [14]:
# Calculate the percentage of ratings 4 or higher for each movie
ratings = df.iloc[:, 2:]
percent_liking = (ratings.ge(4).sum() / ratings.count() * 100).sort_values(ascending=False) #.ge(4) is a boolean mask
percent_liking.head(3)


318: Shawshank Redemption, The (1994)             70.000000
260: Star Wars: Episode IV - A New Hope (1977)    53.333333
3578: Gladiator (2000)                            50.000000
dtype: float64

In [26]:
toy_story = [col for col in df.columns if col.startswith('1: Toy Story')][0] # [0] here means the first column that starts with '1: Toy Story'

raters = df[toy_story].notna()
other_cols = df.columns[2:]

pct = (df.loc[raters, other_cols].notna().sum() / raters.sum() * 100) #(x+y)/x

pct = pct.drop(toy_story)

# Sort by value descending, then by movie id ascending
pct = pct.sort_values(ascending=False)
pct = pct.loc[sorted(pct.index, key=lambda c: ( -pct[c], int(c.split(':')[0]) ))]

top2 = pct.head(4)

# Calculate correlations for the top two movies with Toy Story
correlations = {}
for col in top2.index:
    # Only consider users who rated both movies
    mask = df[toy_story].notna() & df[col].notna()
    if mask.sum() > 1:
        corr = df.loc[mask, [toy_story, col]].corr().iloc[0,1]
    else:
        corr = float('nan')
    correlations[col] = corr

# Display the top 3 as before
display = pct.head(4)
print(display)

# Add the correlations for the top two
print("\nCorrelations with Toy Story for the top two:")
for col in top2.index:
    print(f"{col}: {correlations[col]:.3f}")


260: Star Wars: Episode IV - A New Hope (1977)             82.352941
593: Silence of the Lambs, The (1991)                      76.470588
780: Independence Day (ID4) (1996)                         76.470588
1210: Star Wars: Episode VI - Return of the Jedi (1983)    76.470588
dtype: float64

Correlations with Toy Story for the top two:
260: Star Wars: Episode IV - A New Hope (1977): -0.119
593: Silence of the Lambs, The (1991): -0.568
780: Independence Day (ID4) (1996): -0.070
1210: Star Wars: Episode VI - Return of the Jedi (1983): -0.315


# this shows that not all people who watched and rated same movies liked it equaly and there is actually a negative correlation for the Silence of the Lambs - dislike

In [22]:
toy_story = [c for c in df.columns if c.startswith('1: Toy Story')][0]
other_cols = [c for c in df.columns[2:] if c != toy_story]
corr = {c: df.loc[df[toy_story].notna() & df[c].notna(), [toy_story, c]].corr().iloc[0,1]
        for c in other_cols if df[toy_story].notna().sum() > 1}
corr = pd.Series(corr)
corr = corr[corr > 0].sort_values(ascending=False)
print(corr.head(3))
print("\nCorrelations with Toy Story for the top two:")
for c in corr.head(2).index:
    print(f"{c}: {corr[c]:.3f}")



318: Shawshank Redemption, The (1994)    0.888523
34: Babe (1995)                          0.811107
296: Pulp Fiction (1994)                 0.709842
dtype: float64

Correlations with Toy Story for the top two:
318: Shawshank Redemption, The (1994): 0.889
34: Babe (1995): 0.811


## overlap shows what Toy Story fans also watched (popularity), while correlation shows what they liked in the same way (true taste similarity).

In [28]:
# Which movie do female raters rate the highest above male raters?
# And what are the average ratings for females and males for that movie?

# Identify gender column and movie columns
gender_col = [c for c in df.columns if 'Gender' in c][0]
movie_cols = df.columns[2:]  # assuming first two columns are User and Gender

# Compute mean rating for each movie by gender
mean_by_gender = df.groupby(gender_col)[movie_cols].mean().T

# Compute difference (female mean - male mean) for each movie
mean_diff = mean_by_gender[1] - mean_by_gender[0]  # 1 = F, 0 = M

# Find the movie with the largest difference (female - male)
movie_max_diff = mean_diff.idxmax()
diff_value = mean_diff.loc[movie_max_diff]
female_mean = mean_by_gender.loc[movie_max_diff, 1]
male_mean = mean_by_gender.loc[movie_max_diff, 0]

print(f"Movie where female raters rated highest above male raters:\n{movie_max_diff}")
print(f"Difference (female - male): {diff_value:.3f}")
print(f"Average rating by females: {female_mean:.3f}")
print(f"Average rating by males: {male_mean:.3f}")
# Find movies with greatest positive and negative differences
most_female_above = mean_diff.idxmax()
most_male_above = mean_diff.idxmin()

print("Movie where women rated most above men:")
print(f"{most_female_above}: {mean_diff[most_female_above]:.3f}")

print("\nMovie where men rated most above women:")
print(f"{most_male_above}: {mean_diff[most_male_above]:.3f}")

# Compute overall mean rating for males and females (across all ratings)
male_ratings = df.loc[df[gender_col] == 0, movie_cols].values.flatten()
female_ratings = df.loc[df[gender_col] == 1, movie_cols].values.flatten()

# Remove NaNs
male_ratings = male_ratings[~pd.isna(male_ratings)]
female_ratings = female_ratings[~pd.isna(female_ratings)]

overall_male_mean = male_ratings.mean()
overall_female_mean = female_ratings.mean()
overall_diff = overall_female_mean - overall_male_mean

print(f"\nOverall mean rating (female): {overall_female_mean:.3f}")
print(f"Overall mean rating (male): {overall_male_mean:.3f}")
print(f"Difference (female - male): {overall_diff:.3f}")


Movie where female raters rated highest above male raters:
2396: Shakespeare in Love (1998)
Difference (female - male): 2.107
Average rating by females: 4.250
Average rating by males: 2.143
Movie where women rated most above men:
2396: Shakespeare in Love (1998): 2.107

Movie where men rated most above women:
1198: Raiders of the Lost Ark (1981): -1.667

Overall mean rating (female): 2.947
Overall mean rating (male): 2.906
Difference (female - male): 0.042


On average, male and female ratings look almost identical.
But at the individual movie level, there can be big differences in preference.
Some movies are “gender-neutral” in appeal.
Others (like Shakespeare in Love or Raiders of the Lost Ark) show clear gender splits in taste.

Här är din kod uttryckt som matematiska formler.

**Notation**

* $M$: mängden filmer, index $i\in M$ (filmens id).
* $U_m$, $U_f$: mängden manliga respektive kvinnliga användare, index $u$.
* $r_{u i}$: betyg som användare $u$ gav film $i$; saknas $\Rightarrow$ odefinierat.
* $\mathbf{1}[\cdot]$: indikatorfunktion (1 om sant, annars 0).
* $a_{u i}=\mathbf{1}[r_{u i}\ \text{är definierat}]$ markerar att ett betyg finns.

**Andel 4+ per film och kön**

$$
p_i^{(m)}=\frac{\sum_{u\in U_m}\mathbf{1}[\,r_{u i}\ge 4\,]}{\sum_{u\in U_m} a_{u i}},
\qquad
p_i^{(f)}=\frac{\sum_{u\in U_f}\mathbf{1}[\,r_{u i}\ge 4\,]}{\sum_{u\in U_f} a_{u i}}.
$$

**Skillnad (kvinna − man) per film**

$$
\Delta_i \;=\; p_i^{(f)}-p_i^{(m)}.
$$

**Filmer med största positiva/negativa skillnaden**

$$
i^{+}=\arg\max_{i\in M}\Delta_i,\qquad
i^{-}=\arg\min_{i\in M}\Delta_i.
$$

*(Vid lika värden kan tie-break göras med minsta film-id: välj $ \min\{i:\Delta_i=\max_j \Delta_j\}$, analogt för $\min$).*

**Övergripande andel 4+ per kön (över alla filmer)**

$$
P^{(m)}=\frac{\sum_{u\in U_m}\sum_{i\in M}\mathbf{1}[\,r_{u i}\ge 4\,]}{\sum_{u\in U_m}\sum_{i\in M} a_{u i}},
\qquad
P^{(f)}=\frac{\sum_{u\in U_f}\sum_{i\in M}\mathbf{1}[\,r_{u i}\ge 4\,]}{\sum_{u\in U_f}\sum_{i\in M} a_{u i}}.
$$

**Övergripande skillnad**

$$
\Delta_{\text{overall}} \;=\; P^{(f)}-P^{(m)}.
$$

Detta motsvarar exakt din beräkning: andelar av 4+ per film/kön, differenserna, extremfilmerna samt total skillnad mellan könen.


In [32]:
import re
import pandas as pd

# --- your existing inputs assumed: df, gender_col, movie_cols ---

# For each gender, boolean DataFrame: True if rating >= 4 (ignoring NaN)
male_4plus = (df.loc[df[gender_col] == 0, movie_cols] >= 4)
female_4plus = (df.loc[df[gender_col] == 1, movie_cols] >= 4)

# % of ratings 4+ for each movie (True count / non-NaN count)
male_4plus_pct = male_4plus.sum() / (~df.loc[df[gender_col] == 0, movie_cols].isna()).sum()
female_4plus_pct = female_4plus.sum() / (~df.loc[df[gender_col] == 1, movie_cols].isna()).sum()

# Difference (female % - male %) for each movie
pct_diff = (female_4plus_pct - male_4plus_pct).dropna()

# --- tie-break by lowest movie id ---
def movie_id(col):
    return int(str(col).split(':', 1)[0])

diff_df = pct_diff.to_frame('diff')
diff_df['id'] = diff_df.index.map(movie_id)

# Women > men (max diff; tie -> lowest id)
most_female_above_4plus = diff_df.sort_values(['diff', 'id'], ascending=[False, True]).index[0]
# Men > women (most negative diff; tie -> lowest id)
most_male_above_4plus = diff_df.sort_values(['diff', 'id'], ascending=[True, True]).index[0]

print("Movie where women are most more likely to rate 4+ than men:")
print(f"{most_female_above_4plus}: {pct_diff[most_female_above_4plus]:.3f}")

print("\nMovie where men are most more likely to rate 4+ than women:")
print(f"{most_male_above_4plus}: {pct_diff[most_male_above_4plus]:.3f}")

# --- overall % of 4+ across all movies (build from actual ratings vectors) ---
male_ratings = df.loc[df[gender_col] == 0, movie_cols].stack().dropna()
female_ratings = df.loc[df[gender_col] == 1, movie_cols].stack().dropna()

overall_male_4plus = (male_ratings >= 4).mean()
overall_female_4plus = (female_ratings >= 4).mean()
overall_4plus_diff = overall_female_4plus - overall_male_4plus

print(f"\nOverall % of 4+ ratings (female): {overall_female_4plus:.3f}")
print(f"Overall % of 4+ ratings (male):   {overall_male_4plus:.3f}")
print(f"Difference (female - male):       {overall_4plus_diff:.3f}")


Movie where women are most more likely to rate 4+ than men:
2396: Shakespeare in Love (1998): 0.750

Movie where men are most more likely to rate 4+ than women:
1198: Raiders of the Lost Ark (1981): -0.500

Overall % of 4+ ratings (female): 0.421
Overall % of 4+ ratings (male):   0.339
Difference (female - male):       0.082
